# 如何 利用有偏的硬币作出无偏的决策？

[概率导论（第2版·修订版）](https://book.douban.com/subject/26694188/)的第1章习题33（P55）：  
利用有偏的硬币作出无偏的决策。

题目说明如下：

33\. 利用有偏的硬币作出无偏的决策。爱丽丝和鲍勃想利用一枚均匀的硬币来决定他们去看歌剧还是看电影。  
不幸的是，他们只有一枚有偏的硬币（而且他们并不知道偏的程度）。  
怎样利用一枚有偏的硬币作出无偏的决策，即以1/2的概率看电影，1/2的概率看歌剧呢？

-----------------------

网上的一些讨论：

- [利用有偏硬币做出无偏决策 - niudong - 博客园](https://www.cnblogs.com/andyniu/p/7398901.html)
- [一些有趣的概率问题 - 有偏硬币做出无偏决策 | 零一人生](http://chengfeng96.com/blog/2019/02/25/%E4%B8%80%E4%BA%9B%E6%9C%89%E8%B6%A3%E7%9A%84%E6%A6%82%E7%8E%87%E9%97%AE%E9%A2%98/#%E6%9C%89%E5%81%8F%E7%A1%AC%E5%B8%81%E5%81%9A%E5%87%BA%E6%97%A0%E5%81%8F%E5%86%B3%E7%AD%96)
- [Make a fair coin from a biased coin - GeeksforGeeks](https://www.geeksforgeeks.org/print-0-and-1-with-50-probability/)

# 1. 解法思路说明

偏的硬币有偏的硬币，且不知道偏的程度，是指 

- 抛掷出 正面($H (Head)$)向上 / 反面($T (Tail)$)向上 的概率不是 $1/2$。
- 令正面向上的概率是 $p \in (0, 1)$，即 $P(H) = p$
    - 则反面向上的概率是 $1 - p$, ，即 $P(T) = 1 - p$。
- 但不知道$p$具体值是多少。

如果连续抛2次，实验结果及其概率分布列是：

实验结果:   | $(H, H)$ | $(H, T)$ | $(T, H)$ | $(T, T)$
---------- | --------- | ------- | -------| -------
概率: | $p^2$ | $p(1-p)$ | $(1-p)p$ | $(1-p)^2$

可以发现 实验结果 $(H, T)$ 与 $(T, H)$ 概率相等，都是 $p(1-p)$，与$p$的具体值无关。

利用这一点，可以得到一个 $1/2$ 的概率：

在观察到$(H, T)$ 与 $(T, H)$的条件下，出现$(H, T)$ 或 $(T, H)$的概率是$1/2$。

即 下面的条件概率 是 $1/2$：

$P\big((H, T) \mid \{(H, T), (T, H)\}\big) = P\big((T, H) \mid \{(H, T), (T, H)\}\big) = 1/2$

## 2. 题目答案：利用有偏的硬币作出无偏的决策的操作过程

抛掷2次硬币，以抛掷结果来决定是看歌剧还是看电影。

- 如果结果是 $(H, H)$ 或 $(T, T)$ 则 不算，重新抛掷；
- 如是结果是 $(H, T)$，则 看歌剧；
- 如是结果是 $(T, H)$，则 看电影。

# 3. 通过代码实现及其运行结果验证答案的正确性

## 3.1 代码实现

In [1]:
H = True
T = False

import numpy as np

def count_decision(throw_coin_sequence):
    """
    :param throw_coin_sequence: 抛掷硬币的结果序列
    :return: 返回决策结果，元组`(决策1/HT的次数, 决策2/TH的次数)`
    """
    ht_count = 0 # 决策1/HT的次数
    th_count = 0 # 决策2/TH的次数
    
    idx = 0    
    for _ in throw_coin_sequence:
        if idx % 2 == 1: # 只处理第二次的情况，形成2次抛掷一组的观察
            if throw_coin_sequence[idx] == throw_coin_sequence[idx - 1]:
                # 如果结果是 (H, H) 或 (T, T) 则 不算，重新抛掷
                pass
            elif throw_coin_sequence[idx - 1] == H:
                ht_count += 1
            else: th_count +=1

        idx += 1
                
    return ht_count, th_count


# Unit Test

assert count_decision([H, H]) == (0, 0) # (H,H)，不算作废
assert count_decision([H]) == (0, 0)
assert count_decision([T, T]) == (0, 0) # (T,T)，不算作废
assert count_decision([T]) == (0, 0)

assert count_decision([H, T]) == (1, 0)
assert count_decision([T, H]) == (0, 1)

assert count_decision([
    H, H, # (H,H)，不算作废
    H, T,
    T, T, # (T,T)，不算作废
    T, H
]) == (1, 1)

## 3.2 生成大量的测试数据（1百万次抛掷硬币的结果序列）

In [2]:
p =  1. / 3. # 硬币正面向上的概率 P(H) = 1/3

x = np.random.rand(1_000_000) < p

x

array([False, False,  True, ..., False,  True,  True])

## 3.3 验证生成数据

In [3]:
delta = 0.01

head_frequency = sum(x) / len(x)

assert p - delta < head_frequency < p + delta

head_frequency

0.334423

## 3.4 验证答案是正确的，即决策是公平的

In [4]:
decisions = count_decision(x)

ht_frequency_ratio = decisions[0] / sum(decisions)

assert 0.5 - delta < ht_frequency_ratio < 0.5 + delta 

ht_frequency_ratio, decisions[0], sum(decisions)

(0.4992455396756046, 110839, 222013)

# 4 是不是可以变化操作流程，简化一些呢？简化流程之后还是正确的流程吗？

In [5]:
def count_flipped(xs):
    tf_count = 0
    ft_count = 0
    
    seq_start = True
    v = True
    for x in xs:
        if seq_start:
            seq_start = False
            v = x
            continue

        if v == x:
            continue
        else:
            seq_start = True
            
            if v : tf_count += 1
            else: ft_count += 1
                
    return tf_count, ft_count

In [6]:
count_flipped([True])

(0, 0)

In [7]:
count_flipped([True, False]), \
count_flipped([True,True,True, False]), \
count_flipped([True,True, False, True])

((1, 0), (1, 0), (1, 0))

In [8]:
count_flipped([False, True]), count_flipped([False, False, True])

((0, 1), (0, 1))

In [9]:
count_flipped([True,True,False, False, False, True])

(1, 1)

In [10]:
r = count_flipped(x)

r[0] / sum(r)

0.33521180889160623

## 参考资料

- [在条件概率中，|应该写为\mid - Latex书写误区｜Huan Li's Blog](https://longaspire.github.io/blog/Latex%E4%B9%A6%E5%86%99%E8%AF%AF%E5%8C%BA/)